In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph
from sparse_util import dilate, zhangSuen

In [10]:
date1='0709_1557'
date2='0709_1934'
date3='0709_2357'
plate=13

In [7]:
mat_skel=read_mat(get_path(date1,plate,True))['skel']
dic_skel=dic_to_sparse(mat_skel)
skeleton_doc1=sparse.dok_matrix(dic_skel)
skeleton_doc_1_dilated=dilate(skeleton_doc1)
skeleton_doc_1_thinned=zhangSuen(skeleton_doc_1_dilated)
graph_tab_1_reconnected=from_sparse_to_graph(skeleton_doc_1_thinned)
nx_graph_1,pos_1=generate_nx_graph(graph_tab_1_reconnected)

In [13]:
skeleton_doc_1_dilated=dilate(skeleton_doc1)

In [14]:
skeleton_doc_1_thinned=zhangSuen(skeleton_doc_1_dilated)

1989028 1982302
253320 872
245 14
3 4
1 1
0 0


In [22]:
sparse.save_npz(f'Data/imbackrem_{date1}_{plate}_full',sparse.csc_matrix(skeleton_doc_1_thinned))

In [6]:
skeleton_doc_1_thinned=sparse.dok_matrix(sparse.load_npz(f'Data/imbackrem_{date1}_{plate}_full.npz'))

In [7]:
graph_tab_1_reconnected=from_sparse_to_graph(skeleton_doc_1_thinned)
nx_graph_1,pos_1=generate_nx_graph(graph_tab_1_reconnected)

In [8]:
anx_graph_t0m1=prune_graph(nx_graph_1)

In [11]:
mat_skel=read_mat(get_path(date2,plate,True))['skel']
dic_skel=dic_to_sparse(mat_skel)
skeleton_doc2=sparse.dok_matrix(dic_skel)
skeleton_doc_2_dilated=dilate(skeleton_doc2)
skeleton_doc_2_thinned=zhangSuen(skeleton_doc_2_dilated)
graph_tab_2_reconnected=from_sparse_to_graph(skeleton_doc_2_thinned)
nx_graph_2,pos_2=generate_nx_graph(graph_tab_2_reconnected)
anx_graph_t0=prune_graph(nx_graph_2)

1996689 1989996
252193 1016
360 63
36 32
0 0


In [12]:
sparse.save_npz(f'Data/imbackrem_{date2}_{plate}_full',sparse.csc_matrix(skeleton_doc_2_thinned))

In [22]:
skeleton_doc_2_thinned=sparse.dok_matrix(sparse.load_npz(f'Data/imbackrem_{date2}_{plate}_full.npz'))

In [16]:
def node_dist(node1,node2,nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,show=False):
    #!!! assumed shape == 3000,4096
    sparse_cross1=sparse.dok_matrix((100,100), dtype=bool)
    sparse_cross2=sparse.dok_matrix((100,100), dtype=bool)
    for edge in nx_graph_tm1.edges(node1):
        list_pixel=nx_graph_tm1.get_edge_data(*edge)['pixel_list']
        if (pos_tm1[node1]!=list_pixel[0]).any():
            list_pixel=list(reversed(list_pixel))
        for pixel in list_pixel[:20]:
            sparse_cross1[np.array(pixel)-np.array(pos_tm1[node1])+np.array((50,50))]=1
    for edge in nx_graph_t.edges(node2):
        list_pixel=nx_graph_t.get_edge_data(*edge)['pixel_list']
        if (pos_t[node2]!=list_pixel[0]).any():
            list_pixel=list(reversed(list_pixel))
        for pixel in list_pixel[:20]:
            sparse_cross2[pixel-np.array(pos_tm1[node1])+np.array((50,50))]=1
    kernel = np.ones((3,3),np.uint8)
    dilation1 = cv2.dilate(sparse_cross1.todense().astype(np.uint8),kernel,iterations = 3)
    dilation2 = cv2.dilate(sparse_cross2.todense().astype(np.uint8),kernel,iterations = 3)
    if show:
        plt.imshow(dilation1)
        plt.imshow(dilation2,alpha=0.5)
        plt.show()
    return(np.linalg.norm(dilation1-dilation2))

In [17]:
def first_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t):
    corresp={}
    ambiguous=set()
    to_remove=set()
    degree_3sup_nodes_tm1 = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)>=3]
    degree_3sup_nodes_t = [node for node in nx_graph_t.nodes if nx_graph_t.degree(node)>=3]
    for node1 in degree_3sup_nodes_tm1:
        mini=np.inf
        for node2 in degree_3sup_nodes_t:
            distance=np.linalg.norm(pos_tm1[node1]-pos_t[node2])
            if distance<mini:
                mini=distance
                identifier=node2
        if mini<30:
            if identifier in corresp.values():
                ambiguous.add(node1)
#                     print(node1,'node_dientified_two_times')
            corresp[node1]=identifier
        else:
            to_remove.add(node1)
#                 print(node1,mini,'node_none_iden')
    while len(ambiguous)>0:
        node=ambiguous.pop()
        identifier=corresp[node]
        candidates = [nod for nod in corresp.keys() if corresp[nod]==identifier]
        mini=np.inf
        for candidate in candidates:
            distance=node_dist(candidate,identifier,nx_graph_tm1,nx_graph_t,pos_tm1,pos_t)
            if distance < mini:
                right_candidate=candidate
                mini=distance
#         print(mini,right_candidate)
        for candidate in candidates:
            if candidate!= right_candidate:
                corresp.pop(candidate)
                to_remove.add(candidate)
                ambiguous.discard(candidate)
    return(corresp,to_remove)

In [20]:
from node_id import reconnect_degree_2
def second_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,length_id=50):
    reconnect_degree_2(nx_graph_t,pos_t)
    corresp,to_remove=first_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t)
    nx_graph_tm1=clean_nodes(nx_graph_tm1,to_remove,pos_tm1)
    pos_t,nx_graph_t=relabel_nodes(corresp,nx_graph_t,pos_t)
    corresp_tips={node:node for node in corresp.keys()}
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    for tip in tips:
#         print('tip',pos_tm1[tip],tip)
        mini=np.inf
        for edge in nx_graph_t.edges:
            pixel_list=nx_graph_t.get_edge_data(*edge)['pixel_list']
            if np.linalg.norm(np.array(pixel_list[0])-np.array(pos_tm1[tip]))<=5000:
                distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos_tm1[tip]),axis=1))
                if distance<mini:
                    mini=distance
                    right_edge = edge
        origin = np.array(orient(nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])['pixel_list'],pos_tm1[tip]))
        origin_vector = origin[0]-origin[-1]
        branch=np.array(orient(nx_graph_t.get_edge_data(*right_edge)['pixel_list'],pos_t[right_edge[0]]))
        candidate_vector = branch[-1]-branch[0]
        dot_product = np.dot(origin_vector,candidate_vector)
        if dot_product>=0:
            root=right_edge[0]
            next_node=right_edge[1]
        else:
            root=right_edge[1]
            next_node=right_edge[0]
        last_node=root
        current_node=next_node
        last_branch=np.array(orient(nx_graph_t.get_edge_data(root,next_node)['pixel_list'],pos_t[current_node]))
        while nx_graph_t.degree(current_node)!=1: #Careful : if there is a cycle with low angle this might loop indefinitely but unprobable
            mini=np.inf
            origin_vector = last_branch[0]-last_branch[min(length_id,len(last_branch)-1)]
            unit_vector_origin = origin_vector / np.linalg.norm(origin_vector)
            candidate_vectors=[]
            for neighbours_t in nx_graph_t.neighbors(current_node):
                if neighbours_t!=last_node:
                    branch_candidate=np.array(orient(nx_graph_t.get_edge_data(current_node,neighbours_t)['pixel_list'],pos_t[current_node]))
                    candidate_vector = branch_candidate[min(length_id,len(branch_candidate)-1)]-branch_candidate[0]
                    unit_vector_candidate = candidate_vector / np.linalg.norm(candidate_vector)
                    candidate_vectors.append(unit_vector_candidate)
                    dot_product = np.dot(unit_vector_origin, unit_vector_candidate)
                    angle = np.arccos(dot_product)
                    if angle<mini:
                        mini=angle
                        next_node=neighbours_t
#                     print('angle',dot_product,pos_t[last_node],pos_t[current_node],pos_t[neighbours_t],angle/(2*np.pi)*360)
#!!!bug may happen here if two nodes are direct neighbours : I would nee to check further why it the case, optimal segmentation should avoid this issue.
# This is especially a problem for degree 4 nodes. Maybe fuse nodes that are closer than 3 pixels.
            if len(candidate_vectors)<2:
                print(nx_graph_t.degree(current_node),pos_t[current_node],[node for node in nx_graph_t.nodes if nx_graph_t.degree(node)==2])
            competitor = np.arccos(np.dot(candidate_vectors[0],-candidate_vectors[1]))
#             print('competitor',competitor/(2*np.pi)*360)
            if mini<competitor:
                current_node,last_node=next_node,current_node
            else:
                corresp_tips[tip]=current_node
                break
        corresp_tips[tip]=current_node
    pos_t,nx_graph_t=relabel_nodes(corresp_tips,nx_graph_t,pos_t)
    return(pos_t,nx_graph_t,nx_graph_tm1,corresp_tips)

In [21]:
apos_t0,anx_graph_t0,anx_graph_t0m1,corresp_tips=second_identification(anx_graph_t0m1,anx_graph_t0,pos_1,pos_2,50)

{2051, 4, 2055, 2057, 4107, 4122, 27, 4123, 29, 2087, 6184, 2090, 2091, 2094, 2095, 6193, 6195, 54, 2103, 6198, 4151, 4154, 4150, 6208, 2113, 6210, 2124, 79, 2131, 2134, 2136, 4202, 2155, 4208, 116, 2167, 123, 126, 128, 6276, 133, 4235, 4238, 4239, 4240, 4241, 4242, 4243, 4244, 6287, 4246, 2198, 2199, 153, 4250, 4251, 4252, 6301, 4254, 2206, 158, 2211, 174, 4270, 4274, 4277, 6326, 6330, 188, 6333, 189, 196, 2249, 6354, 4307, 6357, 215, 220, 222, 6366, 4321, 6371, 6373, 6374, 4327, 6376, 6377, 4332, 4338, 2291, 6386, 247, 2296, 4344, 2300, 4351, 255, 6402, 261, 2315, 2328, 2329, 2330, 297, 4394, 301, 4399, 6448, 4403, 4405, 4409, 314, 4412, 6467, 4420, 4421, 2377, 4426, 6475, 333, 334, 2384, 4433, 6483, 6488, 6491, 2395, 2397, 358, 6506, 6510, 2414, 2416, 2417, 2418, 2419, 2420, 2421, 2422, 2423, 2424, 2425, 2426, 2427, 2428, 381, 2430, 6527, 2431, 2432, 2433, 387, 2435, 2436, 2437, 2439, 2440, 2441, 2442, 2438, 4493, 2446, 399, 6543, 403, 413, 4511, 417, 6564, 6575, 2482, 4531, 6579, 2

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
